In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/
!git clone https://github.com/ultralytics/yolov5
%cd yolov5/
%pip install -qr requirements.txt

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 10390, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 10390 (delta 5), reused 7 (delta 4), pack-reused 10380
Receiving objects: 100% (10390/10390), 10.61 MiB | 30.44 MiB/s, done.
Resolving deltas: 100% (7170/7170), done.
/content/yolov5
     |████████████████████████████████| 596 kB 5.2 MB/s 


In [ ]:
### new/pklot Dataset check
from glob import glob

train_img_list = glob('/content/drive/MyDrive/new/camera2/train/images/*.jpg')
test_img_list = glob('/content/drive/MyDrive/new/camera2/test/images/*.jpg')
valid_img_list = glob('/content/drive/MyDrive/new/camera2/valid/images/*.jpg')
print(len(train_img_list), len(test_img_list), len(valid_img_list))

162 39 263


### Customize




In [ ]:
### match dataset


import yaml

with open('/content/drive/MyDrive/new/camera2/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/drive/MyDrive/new/camera2/test.txt', 'w') as f:
  f.write('\n'.join(test_img_list) + '\n')

with open('/content/drive/MyDrive/new/camera2/val.txt', 'w') as f:
  f.write('\n'.join(valid_img_list) + '\n')

In [ ]:
%cat /content/drive/MyDrive/new/camera2/data.yaml


train: /content/drive/MyDrive/new/camera2/train/images
test: /content/drive/MyDrive/new/camera2/test/images
val: /content/drive/MyDrive/new/camera2/valid/images

nc: 5
names: ['person','1','2','empty','car']

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line,cell):
  with open(line,'w') as f:
    f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/drive/MyDrive/new/camera2/data.yaml

train: /content/drive/MyDrive/new/camera2/train/images
test: /content/drive/MyDrive/new/camera2/test/images
val: /content/drive/MyDrive/new/camera2/valid/images

nc: 5
names: ['person','1','2','empty','car']

In [ ]:
%cat /content/drive/MyDrive/new/camera2/data.yaml


train: /content/drive/MyDrive/new/camera2/train/images
test: /content/drive/MyDrive/new/camera2/test/images
val: /content/drive/MyDrive/new/camera2/valid/images

nc: 5
names: ['person','1','2','empty','car']

In [ ]:
with open('/content/drive/MyDrive/new/camera2/data.yaml','r') as stream:
  num_classes = str(yaml.safe_load(stream)['nc'])

%cat /content/yolov5/models/yolov5l.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]], 

In [ ]:
%%writetemplate /content/drive/MyDrive/new/yolov5/models/custom_yolov5l.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
%cat /content/drive/MyDrive/new/yolov5/models/custom_yolov5l.yaml


# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 5  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]], 

### Train

In [ ]:
%time
%cd /content/drive/MyDrive/new/yolov5/
!python train.py --img 640 --batch-size 20 --epochs 50 --data /content/drive/MyDrive/new/camera2/data.yaml --cfg /content/drive/MyDrive/new/yolov5/models/custom_yolov5l.yaml --weights 'yolov5l.pt' --name 2camera --cache

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 9.3 µs
/content/drive/MyDrive/new/yolov5
train: weights=yolov5l.pt, cfg=/content/drive/MyDrive/new/yolov5/models/custom_yolov5l.yaml, data=/content/drive/MyDrive/new/camera2/data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=50, batch_size=20, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=2camera, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 6 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.0-159-gdb6ec66 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf

In [ ]:
### 카메라2번 test 데이터 detect
%cd /content/drive/MyDrive/Colab/yolov5/
!python detect.py --weights /content/drive/MyDrive/new/yolov5/runs/train/2camera13/weights/best.pt --img 640 --conf 0.6 --line-thickness 2 --source /content/drive/MyDrive/new/camera2/test/images/

/content/drive/MyDrive/Colab/yolov5
detect: weights=['/content/drive/MyDrive/new/yolov5/runs/train/2camera13/weights/best.pt'], source=/content/drive/MyDrive/new/camera2/test/images/, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=2, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-155-gdc54ed5 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 367 layers, 46129818 parameters, 0 gradients, 107.9 GFLOPs
tensor(7, device='cuda:0') empty
tensor(2, device='cuda:0') car
image 1/39 /content/drive/MyDrive/new/camera2/test/images/2016-01-14_0804.jpg: 480x640 7 emptys, 2 cars, Done. (0.028s)
tensor(6, device='cuda:0') empty
tensor(4, device='cuda:0') car
image 2/39 /content/drive/MyDrive